In [1]:
!pip install langgraph llama-index langchain langchain-openai python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Sequence, Literal, List, Union
import operator

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool, BaseTool


from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings as LlamaSettings
from llama_index.llms.openai import OpenAI as LlamaOpenAI # LLM para LlamaIndex
from llama_index.embeddings.openai import OpenAIEmbedding as LlamaOpenAIEmbedding # Embeddings para LlamaIndex
from llama_index.core.query_engine import BaseQueryEngine

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

In [7]:
from langchain_ollama.llms import OllamaLLM
from llama_index.embeddings.ollama import OllamaEmbedding
from langchain_ollama.chat_models import ChatOllama
agent_llm = ChatOllama(model="llama3.2:latest")
LlamaSettings.llm = OllamaLLM(model="llama3.2:latest")
LlamaSettings.embed_model =  OllamaEmbedding(model_name="nomic-embed-text")
print("LLMs y dependencias listas.")

LLMs y dependencias listas.


In [8]:
def create_rag_query_engine(data_path: str) -> BaseQueryEngine:
    print(f"Cargando documentos desde: {data_path}")
    documents = SimpleDirectoryReader(data_path).load_data()
    if not documents:
        raise ValueError(f"No se encontraron documentos en {data_path}")
    print(f"Creando índice RAG para {len(documents)} documentos...")
    index = VectorStoreIndex.from_documents(documents)
    query_engine = index.as_query_engine(similarity_top_k=2) # Recuperar 2 fragmentos
    print("Motor de consulta RAG creado.")
    return query_engine

rag_data_path = "../data_rag_agent"
rag_query_engine = create_rag_query_engine(rag_data_path)

# Envolver el motor de consulta RAG en una Langchain Tool
@tool
def rag_search_tool(query: str) -> str:
    """Busca información en la base de conocimiento sobre energías renovables. 
    Úsalo para preguntas específicas sobre energía solar, eólica o geotérmica."""
    print(f"---> Herramienta RAG llamada con la consulta: {query}")
    response = rag_query_engine.query(query)
    return str(response)

tools = [rag_search_tool]
agent_llm_with_tools = agent_llm.bind_tools(tools)

print("Herramienta RAG definida y vinculada al LLM del agente.")

Cargando documentos desde: ../data_rag_agent
Creando índice RAG para 3 documentos...
Motor de consulta RAG creado.
Herramienta RAG definida y vinculada al LLM del agente.


In [9]:
class ResearchAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    original_question: str
    research_steps: Annotated[List[str], operator.add]
    final_answer: Union[str, None]

In [10]:
# Nodo 1: Agente Planificador/Decisor
def planner_agent_node(state: ResearchAgentState):
    print("---NODO: planner_agent_node---")
    # El prompt podría ser más sofisticado, indicando que ya ha intentado X pasos
    current_messages = state["messages"]
    if not any(isinstance(msg, SystemMessage) for msg in current_messages):
        # Añadir un SystemMessage si no existe para guiar mejor al LLM
        system_prompt = SystemMessage(content=(
            "Eres un asistente de investigación. Tu objetivo es responder a la pregunta del usuario. "
            "Primero intenta responder desde tu conocimiento general. Si no puedes o necesitas detalles específicos "
            "sobre energías renovables (solar, eólica, geotérmica), debes usar la herramienta 'rag_search_tool'. "
            "Después de usar la herramienta, evalúa si tienes suficiente información o si necesitas volver a usarla. "
            "Una vez que tengas suficiente información, indica que estás listo para dar una respuesta final."
        ))
        current_messages = [system_prompt] + list(current_messages)

    response = agent_llm_with_tools.invoke(current_messages)
    return {"messages": [response], "research_steps": [f"Planner LLM: {response.content}"]}

# Nodo 2: Ejecutor de la herramienta RAG (usando ToolNode preconstruido)
rag_tool_node = ToolNode([rag_search_tool]) # Pasamos la lista de herramientas

# Nodo 3: Nodo para generar la respuesta final (opcional, o el planner puede hacerlo)
# Por simplicidad, dejaremos que el planner_agent_node también maneje la generación de la respuesta final
# cuando decida que tiene suficiente información (es decir, no llama a una herramienta).

print("Nodos del grafo definidos.")

Nodos del grafo definidos.


In [14]:
MAX_ITERATIONS = 3 # Para evitar bucles infinitos

def should_use_rag_or_end(state: ResearchAgentState) -> Literal["use_rag_tool", "end_research"]:
    print("---NODO CONDICIONAL: should_use_rag_or_end---")
    last_message = state["messages"][-1]
    current_iteration = len([step for step in state.get("research_steps", []) if "Planner LLM:" in step or "Tool Result:" in step])

    if current_iteration >= MAX_ITERATIONS:
        print(f"Límite de iteraciones ({MAX_ITERATIONS}) alcanzado. Finalizando.")
        return "end_research"

    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        # Específicamente verificar si la herramienta es rag_search_tool
        if any(tc["name"] == rag_search_tool.name for tc in last_message.tool_calls):
            print("Decisión: LLM solicitó la herramienta RAG. Ir a 'use_rag_tool'.")
            return "use_rag_tool"
    
    print("Decisión: LLM no solicitó la herramienta RAG o es una respuesta. Finalizando para síntesis.")
    return "end_research"

print("Lógica condicional definida.")

Lógica condicional definida.


In [15]:
workflow = StateGraph(ResearchAgentState)

workflow.add_node("planner_agent", planner_agent_node)
workflow.add_node("rag_tool_executor", rag_tool_node)

workflow.set_entry_point("planner_agent")

workflow.add_conditional_edges(
    "planner_agent",
    should_use_rag_or_end,
    {
        "use_rag_tool": "rag_tool_executor",
        "end_research": END 
    }
)

# Después de ejecutar la herramienta RAG, volvemos al agente planificador para que evalúe
workflow.add_edge("rag_tool_executor", "planner_agent")

app = workflow.compile()
print("Grafo de investigación compilado.")

Grafo de investigación compilado.


In [16]:
initial_question = "¿Cuáles son las ventajas de la energía solar según la base de conocimiento?"
inputs = {
    "messages": [HumanMessage(content=initial_question)],
    "original_question": initial_question,
    "research_steps": [],
    "final_answer": None
}

print(f"\n--- Iniciando Agente de Investigación para: {initial_question} ---")

# Usar stream para ver los eventos y el estado en cada paso
for event in app.stream(inputs, {"recursion_limit": 10}):
    for key, value in event.items():
        print(f"Evento del Grafo: Nodo='{key}'")
        # print(f"Estado actualizado: {value}") # Puede ser muy verboso
        if "messages" in value:
            print(f"  Último mensaje: {value['messages'][-1].pretty_repr()[:300]}...")
        if "research_steps" in value and value["research_steps"]:
            print(f"  Último paso de investigación: {value['research_steps'][-1]}")
    print("---")

final_state = app.invoke(inputs, {"recursion_limit": 10})
final_llm_response_message = final_state["messages"][-1]

print("\n--- Resultado Final de la Investigación ---")
print(f"Pregunta Original: {final_state['original_question']}")
print(f"Respuesta del Agente: {final_llm_response_message.content}")



--- Iniciando Agente de Investigación para: ¿Cuáles son las ventajas de la energía solar según la base de conocimiento? ---
---NODO: planner_agent_node---
---NODO CONDICIONAL: should_use_rag_or_end---
Decisión: LLM solicitó la herramienta RAG. Ir a 'use_rag_tool'.
Evento del Grafo: Nodo='planner_agent'
  Último mensaje: ================================== Ai Message ==================================
Tool Calls:
  rag_search_tool (d0cbf774-8b1c-4e2d-8bfe-977600b417d3)
 Call ID: d0cbf774-8b1c-4e2d-8bfe-977600b417d3
  Args:
    query: ventajas de la energía solar...
  Último paso de investigación: Planner LLM: 
---
---> Herramienta RAG llamada con la consulta: ventajas de la energía solar
Evento del Grafo: Nodo='rag_tool_executor'
  Último mensaje: ================================= Tool Message =================================
Name: rag_search_tool

Las ventajas de la energía solar son:

*   Es una fuente limpia y abundante, lo que reduce la contaminación del medio ambiente.
*   La ener